In [1]:
import time
import os
import sys
import logging

import numpy as np
import tenseal as ts

from sklearn.model_selection import train_test_split
os.chdir("..")
from src.features import build_features
from models import encrypted_LR

# definition of parameters

In [14]:
# data_path = "~/homomorphic-encryption/ckks_titanic/data/raw/"            # whole data set
DATA_PATH = "~/homomorphic-encryption/ckks_titanic/data/quick_demo/"   # subset of the data set, with 15 train_samples and 5 test_samples
LOG_PATH = "."
LOG_FILENAME = "date+all"

In [15]:
fileHandler = logging.FileHandler("{0}/{1}.log".format(LOG_PATH, LOG_FILENAME))
streamHandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(format="%(asctime)s  [%(levelname)-8.8s]  %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.DEBUG, handlers=[fileHandler, streamHandler])

In [56]:
def crytp_array(X, local_context):
    res = []
    for vec in X:
        res.append(ts.ckks_vector(local_context, vec))
    return res

def refresh(ciphertext, **kwarg):
    context = kwarg.get("context", None)
    secret_key = kwarg.get("secret_key", None)
    assert context , "Context must be provided with the key : context"+str(context)
    if context.is_private():
        return ts.ckks_vector(context, ciphertext.decrypt())
    else : 
        if not secret_key:
            raise AttributeError("The secret key is not provided and the context provided is public, decryption is not possible")
        else:
            return ts.ckks_vector(context, ciphertext.decrypt(secret_key))

# Loading and processing the data

In [17]:
logging.info(os.getcwd())
raw_train, raw_test = build_features.data_import(DATA_PATH)
train, submission_test = build_features.processing(raw_train, raw_test)

07/15/2020 12:01:58 PM  [INFO    ]  /home/apignet/homomorphic-encryption/ckks_titanic
07/15/2020 12:01:58 PM  [INFO    ]  loading the data into memory (pandas df)
07/15/2020 12:01:58 PM  [INFO    ]  Done
07/15/2020 12:01:58 PM  [INFO    ]  making final data set from raw data
07/15/2020 12:01:58 PM  [INFO    ]  Done


In [18]:
train, test = train_test_split(train, test_size=0.15)
train_labels = train.Survived
test_labels = test.Survived
train_features = train.drop("Survived", axis=1)
test_features = test.drop("Survived", axis=1)

# Definition of safety parameters

In [52]:
logging.info('Definition of safety parameters...')
timer = time.time()
# context = ts.context(ts.SCHEME_TYPE.CKKS, 32768,
# coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60])
# context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40])

context = ts.context(ts.SCHEME_TYPE.CKKS, 16384, coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 60])
context.global_scale = pow(2, 40)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")
logging.info('Generation of the Galois Key...')
timer = time.time()
context.generate_galois_keys()
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 01:26:30 PM  [INFO    ]  Definition of safety parameters...
07/15/2020 01:26:30 PM  [INFO    ]  Done. 0.47 seconds
07/15/2020 01:26:30 PM  [INFO    ]  Generation of the Galois Key...
07/15/2020 01:26:35 PM  [INFO    ]  Done. 4.89 seconds


# Data encryption

In [20]:
logging.info("Data encryption...")
timer = time.time()
encrypted_X = crytp_array(train_features.to_numpy(), context)
encrypted_Y = crytp_array(train_labels.to_numpy().reshape((-1, 1)), context)
encrypted_test_X = crytp_array(test_features.to_numpy(), context)
encrypted_test_Y = crytp_array(test_labels.to_numpy().reshape((-1, 1)), context)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 12:02:18 PM  [INFO    ]  Data encryption...
07/15/2020 12:02:19 PM  [INFO    ]  Done. 1.24 seconds


# Training

In [22]:
logging.info("Training starting")
model = encrypted_LR.LogisticRegressionHE(local_context=context, safety=False, verbose=True, num_iter=40)
model.fit(encrypted_X, encrypted_Y, X_ne=train_features.to_numpy(), Y_ne=train_labels.to_numpy().reshape((-1, 1)))
logging.info("Training done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 12:02:40 PM  [INFO    ]  Training starting
07/15/2020 12:02:40 PM  [CRITICAL]   The data will be decrypted during the process, the protocol is not safe
07/15/2020 12:02:52 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:02:52 PM  [DEBUG   ]  [-0.0008985056047738081, -0.0008589436430576638, -0.0008333454061351996, 0.0025000395508974257, -0.0007382240789089848, -0.0004166693179446673, -0.0004166729877219238, 0.001666699391810221, -0.0008333457841133833, 0.0004166695360241519, 0.0012500112925205012, 0.0012500216037588468, -0.000833342688106731, -1.5034372948688294e-09, 0.0004166733723439574, -7.176739406944608e-10, -5.473999335439029e-09, -3.466639163171946e-09, -6.734339985035301e-10, 5.644429384172372e-09, -0.000833347911211258, 9.501742630853192e-10, 0.00041667363503898407, -5.367281278191503e-10, -0.0004166730527547971, 0.0016667029701874468, 0.0004166711851051973, -0.00041666974151174143, 0.0020833627237473173, -0.0012500182793624985, -1.5630147336992193e-09]
07/15/2020 12:02

07/15/2020 12:03:53 PM  [DEBUG   ]  error 0
07/15/2020 12:04:06 PM  [INFO    ]  iteration number 6 is starting
07/15/2020 12:04:06 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:04:06 PM  [DEBUG   ]  [-0.0008665811370323526, -0.0008308346839209599, -0.0008464645421671149, 0.0024455818195065655, -0.00073207815918946, -0.0004174220553108945, -0.00041592947162553506, 0.0016075233492686778, -0.0008279940424407905, 0.00040640587800491105, 0.0011957355745577125, 0.0012033999142658813, -0.0008382300124700577, -9.568707579821019e-10, 0.0004089850600165826, -4.524801375954297e-10, -1.7486348250930407e-09, 2.8547577164034497e-09, 3.1336359220312623e-09, -3.329478737131136e-09, -0.0008280543969667067, -3.026110153501288e-09, 0.0004106271405927857, -1.9185039994909658e-10, -0.00041793168594633, 0.0016095263455576736, 0.00040898402537986513, -0.00042466609266048256, 0.002036603509108106, -0.0012467513976042513, 1.795693665218248e-09]
07/15/2020 12:04:06 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:

07/15/2020 12:05:07 PM  [DEBUG   ]  error 0
07/15/2020 12:05:19 PM  [INFO    ]  iteration number 12 is starting
07/15/2020 12:05:19 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:05:19 PM  [DEBUG   ]  [-0.0008302368436432252, -0.0007984632616106095, -0.0008608267798801128, 0.0023826349094760698, -0.0007244615996958215, -0.0004179990645076982, -0.0004148811746242793, 0.001539670273315716, -0.0008214202176702539, 0.0003945091624994885, 0.0011336899534499399, 0.0011498341215025391, -0.0008432194492464891, 4.014471753676341e-09, 0.0004001752262893872, 3.1542996397297065e-10, 4.651167911091644e-09, 7.451806891687956e-10, -4.630827512459954e-09, 1.9701162037724273e-09, -0.0008216007136123689, -2.8819507564427367e-09, 0.0004036063137506062, -2.9879146479363764e-10, -0.0004191539873853475, 0.001543952326291285, 0.0004001780818409663, -0.00043349433591477937, 0.0019824591866764245, -0.0012423408358916686, -1.185344488477413e-09]
07/15/2020 12:05:19 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:0

07/15/2020 12:06:20 PM  [DEBUG   ]  error 0
07/15/2020 12:06:32 PM  [INFO    ]  iteration number 18 is starting
07/15/2020 12:06:32 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:06:32 PM  [DEBUG   ]  [-0.0007959317350754928, -0.0007675160839509527, -0.0008737836211081475, 0.0023222049319132658, -0.0007166241823390384, -0.0004182203070821076, -0.0004136757366885891, 0.0014751393478639554, -0.0008146936537807078, 0.0003830507620388982, 0.0010748644256394946, 0.0010987736208281766, -0.0008473470086604282, 1.930424253466231e-09, 0.0003918032061166161, -2.993945102095459e-10, -4.082159677214135e-09, 3.306732667291234e-09, 5.521506208102518e-10, 2.968792755758447e-09, -0.0008150539446158604, -4.850264840904398e-09, 0.000396825121405284, 2.1017200251545935e-09, -0.0004200926052773425, 0.001481556163000342, 0.00039180564802145714, -0.00044152667099687593, 0.0019303918178700612, -0.0012374376704983171, 2.338618567823011e-09]
07/15/2020 12:06:32 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:06:3

07/15/2020 12:07:33 PM  [DEBUG   ]  error 0
07/15/2020 12:07:45 PM  [INFO    ]  iteration number 24 is starting
07/15/2020 12:07:45 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:07:45 PM  [DEBUG   ]  [-0.0007635324580595102, -0.0007379498926495445, -0.0008854069894623166, 0.0022641902036668413, -0.0007085900737927428, -0.00041811408032860784, -0.0004123269162655244, 0.0014137583174218186, -0.000807828968012641, 0.00037201649529702765, 0.0010191060998651848, 0.001050102838010308, -0.0008506437730842198, 1.917240540376946e-09, 0.0003838399961277519, 1.6428612000630145e-10, -1.5557651611662943e-09, -3.3467180061799263e-09, 1.2380092876695484e-09, -4.3260934350597485e-09, -0.0008084156818026862, 9.301068067319763e-11, 0.0003902943799524345, 3.6951403506727115e-09, -0.00042075573319612554, 0.0014221909004682688, 0.0003838362655370695, -0.0004488103341881099, 0.001880344583872704, -0.001232074947569248, 4.973836822975004e-10]
07/15/2020 12:07:45 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:

07/15/2020 12:08:46 PM  [DEBUG   ]  error 0
07/15/2020 12:08:58 PM  [INFO    ]  iteration number 30 is starting
07/15/2020 12:08:58 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:08:58 PM  [DEBUG   ]  [-0.0007329658338464556, -0.0007096795036285214, -0.0008957812951366779, 0.0022084952989214167, -0.0007003985122415474, -0.0004177212610493794, -0.00041083190828176085, 0.0013553800194562068, -0.000800851106864476, 0.00036139563045174553, 0.0009662692044749258, 0.001003723883918106, -0.0008531587112370944, 1.7890971126231704e-09, 0.0003762645009113341, -6.370274228514445e-10, -1.0454966585430692e-09, 1.2802738153809883e-09, -1.3936891428715644e-09, 8.368484109093264e-09, -0.0008016939161639791, 1.1901199593354682e-10, 0.00038397262605937134, -1.996177882185814e-10, -0.000421164929593198, 0.00136571811891085, 0.0003762634622571371, -0.0004553841555067971, 0.0018322226492305405, -0.0012262852484851055, -3.979422766959092e-09]
07/15/2020 12:08:58 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:

07/15/2020 12:09:59 PM  [DEBUG   ]  error 0
07/15/2020 12:10:11 PM  [INFO    ]  iteration number 36 is starting
07/15/2020 12:10:11 PM  [DEBUG   ]  encrypted grad
07/15/2020 12:10:11 PM  [DEBUG   ]  [-0.0007041237041729106, -0.000682668991144072, -0.0009049640449293822, 0.0021550216981380318, -0.0006920605041580048, -0.00041703428625875904, -0.0004092164027676052, 0.0012998836563216966, -0.0007937595769649975, 0.00035116321019114334, 0.000916212889889058, 0.0009595324921757606, -0.0008549623126645903, 1.009449942744096e-09, 0.0003690530564851514, -2.7872914986160965e-10, 1.7380847715597696e-11, -6.4089743012811735e-09, -1.5357585090877822e-09, 5.873782252830287e-09, -0.0007949026196482644, -1.5709473269435748e-09, 0.0003778643632582116, 1.938946482147212e-09, -0.00042134261803606363, 0.001312013412008773, 0.0003690548028500683, -0.000461288356865158, 0.0017859529614529033, -0.0012200997191247776, 7.988986711585002e-10]
07/15/2020 12:10:11 PM  [DEBUG   ]  Unencrypted grad
07/15/2020 12:

# Accuracy

In [23]:
acc = model.encrypted_accuracy(encrypted_test_X, encrypted_test_Y)
logging.info("Accuracy : %s " % acc[0])

ValueError: scale out of bounds